# Spark SQL Magic Demo

##Setup
The `sparksql` magic is an easy way to run Spark SQL from a notebook. It interacts with a pyspark SQLContext, so it is required that pyspark is properly installed and configured.

In [1]:
from pyspark import SparkContext, SQLContext
sparkcon = SparkContext(appName="demo")
sqlcon = SQLContext(sparkcon)

`%load_ext` is used to initialize the sparksql magic.

In [2]:
%load_ext sparksql

## Loading Data

All data used in this demo is example data from the spark project.

### Loading Local files
`sparksql` can load local files using the `-l` option. Note that the magic automatically finds the SQLContext in the user namespace. Currently supported file formats are JSON and parquet.

In [3]:
%sparksql -l data/people.json

Stored data/people.json in table people


,tableName,isTemporary
0,people,True


In [4]:
%sparksql -l data/users.parquet

Stored data/users.parquet in table users


,tableName,isTemporary
0,users,True
1,people,True


## Querying Data

`sparksql` can run SQL queries against the loaded data.

In [5]:
%sparksql SELECT * FROM users

,name,favorite_color,favorite_numbers
0,Alyssa,None,"[3, 9, 15, 20]"
1,Ben,red,[]


The data is returned as a pyspark DataFrame and can be stored and transformed.

In [6]:
names = %sparksql SELECT name FROM people
names = names.map(lambda n: "Name: " + n.name + "    Length: " + str(len(n.name)))
for name in names.collect():
    print(name)

Name: Michael    Length: 7
Name: Andy    Length: 4
Name: Justin    Length: 6


Queries can span multiple lines if the cell magic is used.

In [7]:
%%sparksql
SELECT name, age
FROM people
WHERE age < 20;

,name,age
0,Justin,19


The cell magic also allows for running multiple queries, but only the final query's result is displayed.

In [8]:
%%sparksql
SELECT name, age
FROM people;
SHOW TABLES;

,tableName,isTemporary
0,users,True
1,people,True


## Injecting Variables

Local variables can be placed in the query by referencing them prepended with a colon. The string representation of the variable will be used.

In [9]:
max_age = 21
%sparksql SELECT name, age FROM people WHERE age < :max_age

,name,age
0,Justin,19


In [10]:
column = "name"
%sparksql SELECT :column FROM users

,name
0,Alyssa
1,Ben


If a variable is to be used as a string literal, it must be enclosed in quotes.

In [11]:
name = "Ben"
column = "name"
%sparksql SELECT :column FROM users WHERE name=":name"

,name
0,Ben


## Serializing Data
The result of a query can be saved to a local JSON or parquet file using the -w flag. The format to write in is inferred from the file extension.

In [12]:
%%sparksql -w output/names.json
SELECT name
FROM users;

,name
0,Alyssa
1,Ben


Note that the output will be a directory containing the individual parts of the output.

In [13]:
! ls output/names.json

part-00000  _SUCCESS


In [14]:
! rm -r output/names.json

## Multiple Contexts

If there are multiple SQLContexts present in the user namespace, the last one used by the magic will be preferred.

In [15]:
sqlcon2 = SQLContext(sparkcon)
# Uses original sqlcon
%sparksql SHOW TABLES;

,tableName,isTemporary
0,users,True
1,people,True


A different context can be specified using the `-s` option.

In [16]:
#Uses the new sqlcon2
%sparksql -s sqlcon2 SHOW TABLES;

,tableName,isTemporary
